In [1]:
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" pip install llama-cpp-python

In [2]:
!pip install huggingface-hub

In [3]:
!huggingface-cli download TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF capybarahermes-2.5-mistral-7b.Q4_K_M.gguf --local-dir . --local-dir-use-symlinks False

Consider using `hf_transfer` for faster downloads. This solution comes with some limitations. See https://huggingface.co/docs/huggingface_hub/hf_transfer for more details.
downloading https://huggingface.co/TheBloke/CapybaraHermes-2.5-Mistral-7B-GGUF/resolve/main/capybarahermes-2.5-mistral-7b.Q4_K_M.gguf to /root/.cache/huggingface/hub/tmp2dn5_9xa
(…)apybarahermes-2.5-mistral-7b.Q4_K_M.gguf: 100% 4.37G/4.37G [00:36<00:00, 119MB/s]
./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf


In [96]:
def generate_sentences_and_storyboard(context):
  from llama_cpp import Llama

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message="you are now a marketing designer for a video to promote a given policy. Come up with scenes and describe each scene with singaporean context and simple english and clearly on the shot also give content from the policy given to be narrated over. For example be specific Asian Family, Singaporean. Total minimum 25 scenes and 10 word max per sentence be clear and precise, the output format should strictly be Scene: Content: "
  prompt = context
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=1024,  # Generate up to 1024 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  list = texts.split('\n')
  print(texts)
  storyboard = []
  clean_sentences = []
  narration = []
  for sentence in list:
    if 'Scene: ' in sentence:
      clean_sentences.append(sentence.split('Scene: ')[1])
      storyboard.append(sentence)
    elif 'Content: ' in sentence:
      narration.append(sentence.split('Content: ')[1])
      storyboard.append(sentence)

  return clean_sentences, storyboard, narration

In [68]:
def generate_text_for_voiceover(data):
  from llama_cpp import Llama
  no_of_sentences=len(data)
  # Concatenated the sentences:
  combined_sentences=''
  for sentence in data:
    combined_sentences+= sentence + '\n'

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message=f"You are required to make a script for narration based on the context given without changing the facts, the output will be a paragraph that is {no_of_sentences*10} words speech do not exceed the word limit."
  prompt = combined_sentences
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  return texts

In [69]:
def generate_title_and_tagline(context):
  from llama_cpp import Llama

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message="You are required to generate a title and a tagline based on the content given"
  prompt = context
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  texts = texts.split('\n')
  for sentence in texts:
    if 'Title: ' in sentence:
      # Note that the title has ""
      title=sentence.split('Title: ')[1].replace('"','')
    elif 'Tagline: ' in sentence:
      tagline=sentence.split('Tagline: ')[1].replace('"','').rstrip('.')
  return tagline, title

In [70]:
def generate_theme_for_music(context):
  from llama_cpp import Llama

  # Set gpu_layers to the number of layers to offload to GPU. Set to 0 if no GPU acceleration is available on your system.
  llm = Llama(
    model_path="./capybarahermes-2.5-mistral-7b.Q4_K_M.gguf",  # Download the model file first
    n_ctx=32768,  # The max sequence length to use - note that longer sequence lengths require much more resources
    n_threads=8,            # The number of CPU threads to use, tailor to your system and the resulting performance
    n_gpu_layers=35         # The number of layers to offload to GPU, if you have GPU acceleration available
  )
  # Simple inference example
  system_message="You are required to generate a music genre name fit for a background music over the content. The output will be the genre only."
  prompt = context
  output = llm(
    f"<|im_start|>system\n{system_message}<|im_end|>\n<|im_start|>user\n{prompt}<|im_end|>\n<|im_start|>assistant", # Prompt
    max_tokens=512,  # Generate up to 512 tokens
    stop=["</s>"],   # Example stop token - not necessarily correct for this specific model! Please check before using.
    echo=False        # Whether to echo the prompt
)
  texts = output['choices'][0]['text']
  texts = texts.split('\n')

  for text in texts:
    if text !='':
      genre= text
  return genre

In [97]:
context = """Singapore will launch a series of policies in the coming months to reduce the impact of inflation on people’s cost of living, including new workplace family supportive policies, according to its Budget for 2023.

Mayumi Soh of Pinsent Masons MPillay, the Singapore joint law venture between MPillay and Pinsent Masons, said: “Rising costs of living poses a serious concern for many Singaporeans amidst global uncertainty, and the Singapore government’s Budget 2023 seeks to introduce measures to alleviate some of these concerns.”

According to the Budget announcement, Singapore’s government-paid paternity leave will be doubled from two weeks to four weeks from 1 January 2024. In the meantime, employers will retain the option to grant the two extra weeks of leave voluntarily and any additional leave granted will be reimbursed by the government accordingly. Unpaid infant care leave will be increased from six days to 12 days a year for each parent in the child’s first two years.

The baby bonus cash gift will also be increased by S$3,000 for each eligible Singaporean child born from 14 February 2024 – the bonus will be paid every six months until the child turns six-and-a-half years old. The Child Development Account (CDA) First Step Grant will be increased from S$3,000 to S$5,000.

The Central Provident Fund (CPF) monthly salary ceiling will be increased in stages from S$6,000 to S$8,000 by 2026. From September, it will be raised from S$6,000 to S$6,300 and will subsequently increase to S$6,800 from January 2024; S$7,400 from January 2025; and S$8,000 from January 2026. The CPF annual salary ceiling will remain the same at S$102,000. The minimum CPF monthly payout for seniors on the Retirement Sum Scheme will be raised from S$250 to S$350 effective from 1 June.

The government will also provide a one-time cost-of-living special payment to Singaporeans aged 21 and above in 2023 who have an annual assessable income below S$100,000 and who do not own more than one property. Singaporeans who earn S$22,000 each year will receive S$400; those who earn between S$22,000 and S$34,000 will receive S$300; and those who earn over S$34,000 and up to S$100,000 will receive S$200. The money will be distributed in June.

Also, the government will increase the home grant by up to S$30,000 for families who buy resale flats for the first time based on the flat size. Families will only be eligible for the grant if they do not earn over S$14,000 monthly and do not own any private property.

Lastly, there will be a higher marginal buyer’s stamp duty applied to all properties bought from 15 February. For residential properties, the portion of the property value over S$1.5 million and up to S$3m will be taxed at 5%, while the part over S$3m will be taxed at 6%, up from the existing rate of 4%. For non-residential properties, the part of the property value over S$1m and up to S$1.5m will be taxed at 4%, while the part over S$1.5m will be taxed at 5%, up from existing 3%. The initiative seeks to target higher-value properties as part of an enhanced property cooling measure.
"""

In [98]:
data, story, narration = generate_sentences_and_storyboard(context)

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '32000', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'argilla_capybarahermes-2.5-mistral-7b', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_


Scene: A working-class Singaporean family having dinner at home.
Content: Inflation is affecting people's cost of living in Singapore.

Scene: A mother struggling with groceries in a supermarket, looking concerned.
Content: The government's Budget 2023 aims to alleviate these concerns.

Scene: Mayumi Soh of Pinsent Masons MPillay explaining the measures in the Budget announcement.
Content: New workplace family supportive policies are introduced.

Scene: A father holding a newborn baby in their home.
Content: Paternity leave will be doubled from 2 weeks to 4 weeks starting 1 January 2024.

Scene: A father at work, talking on the phone.
Content: Employers can choose to grant additional two weeks of leave, reimbursed by the government.

Scene: A mother caring for her baby at home.
Content: Unpaid infant care leave increased from 6 days to 12 days per year for each parent in a child's first two years.

Scene: A young family receiving a check in celebration of their new baby.
Content: Baby

In [99]:
tagline, title = generate_title_and_tagline(context)
summary = generate_text_for_voiceover(narration)
genre=generate_theme_for_music(context)
string = f'{tagline}, with title "{title}" on it,Movie Poster, MoviePosterRedAF,,'

AVX = 1 | AVX_VNNI = 0 | AVX2 = 1 | AVX512 = 1 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 1 | VSX = 0 | 
Model metadata: {'tokenizer.chat_template': "{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}", 'tokenizer.ggml.add_eos_token': 'false', 'tokenizer.ggml.padding_token_id': '0', 'tokenizer.ggml.unknown_token_id': '0', 'tokenizer.ggml.eos_token_id': '32000', 'general.architecture': 'llama', 'llama.rope.freq_base': '10000.000000', 'llama.context_length': '32768', 'general.name': 'argilla_capybarahermes-2.5-mistral-7b', 'tokenizer.ggml.add_bos_token': 'true', 'llama.embedding_length': '4096', 'llama.feed_forward_length': '14336', 'llama.attention.layer_norm_rms_epsilon': '0.000010', 'llama.rope.dimension_count': '128', 'tokenizer.ggml.bos_

In [81]:
string

'Government introduces measures including extended paternity leave, increased baby bonus, and higher home grants while implementing higher marginal buyer\'s stamp duty for higher-value properties, with title "Singapore\'s 2023 Budget: Enhancing Family Support and Combat Inflation through Policy Updates" on it,Movie Poster, MoviePosterRedAF,,'

In [100]:
narration

["Inflation is affecting people's cost of living in Singapore.",
 "The government's Budget 2023 aims to alleviate these concerns.",
 'New workplace family supportive policies are introduced.',
 'Paternity leave will be doubled from 2 weeks to 4 weeks starting 1 January 2024.',
 'Employers can choose to grant additional two weeks of leave, reimbursed by the government.',
 "Unpaid infant care leave increased from 6 days to 12 days per year for each parent in a child's first two years.",
 'Baby bonus cash gift increased by S$3,000 per eligible child.',
 'CDA First Step Grant increased from S$3,000 to S$5,000.',
 'CPF monthly salary ceiling increased in stages to S$8,000 by 2026.',
 'Minimum CPF monthly payout for seniors raised from S$250 to S$350, effective 1 June.',
 'One-time cost-of-living special payment distributed to eligible Singaporeans in June 2023.',
 'Increased home grant of up to S$30,000 for first-time buyers of resale flats.',
 "Higher marginal buyer's stamp duty applied to

In [101]:
summary

"\nInflation has been a significant concern for individuals in Singapore, impacting their daily expenses. To alleviate these issues, the government has crafted the Budget 2023, which aims to provide relief and support for families. A key part of this is the introduction of new workplace family supportive policies. From January 2024, paternity leave will be doubled from two weeks to four weeks, with employers having the option to grant additional two weeks of leave reimbursed by the government. Parental leave has also been increased from six days per year to twelve days per year for each parent during their child's first two years.\n\nTo further support families, the baby bonus cash gift has increased by S$3,000 per eligible child, and the CDA First Step Grant has been raised from S$3,000 to S$5,000. In addition, the CPF monthly salary ceiling has been increased in stages up to S$8,000 by 2026. The minimum CPF monthly payout for seniors has also been raised from S$250 to S$350 starting 

In [102]:
data

['A working-class Singaporean family having dinner at home.',
 'A mother struggling with groceries in a supermarket, looking concerned.',
 'Mayumi Soh of Pinsent Masons MPillay explaining the measures in the Budget announcement.',
 'A father holding a newborn baby in their home.',
 'A father at work, talking on the phone.',
 'A mother caring for her baby at home.',
 'A young family receiving a check in celebration of their new baby.',
 'A child standing in front of a CDA First Step Grant banner.',
 'A middle-aged couple discussing finances at their kitchen table.',
 'A senior couple holding hands, smiling at camera.',
 'A young adult receiving a letter in the mail.',
 'A young couple visiting an open house for resale flats.',
 'A family looking at a property with a real estate agent.',
 'A working-class family celebrating together at home, smiling at camera.']

In [103]:
genre

'Ambient Policy Pop'

In [104]:
story

['Scene: A working-class Singaporean family having dinner at home.',
 "Content: Inflation is affecting people's cost of living in Singapore.",
 'Scene: A mother struggling with groceries in a supermarket, looking concerned.',
 "Content: The government's Budget 2023 aims to alleviate these concerns.",
 'Scene: Mayumi Soh of Pinsent Masons MPillay explaining the measures in the Budget announcement.',
 'Content: New workplace family supportive policies are introduced.',
 'Scene: A father holding a newborn baby in their home.',
 'Content: Paternity leave will be doubled from 2 weeks to 4 weeks starting 1 January 2024.',
 'Scene: A father at work, talking on the phone.',
 'Content: Employers can choose to grant additional two weeks of leave, reimbursed by the government.',
 'Scene: A mother caring for her baby at home.',
 "Content: Unpaid infant care leave increased from 6 days to 12 days per year for each parent in a child's first two years.",
 'Scene: A young family receiving a check in c

In [107]:
import pickle

In [108]:
# Specify file paths for each variable
story_pickle_file = 'story_data.pickle'
genre_pickle_file = 'genre_data.pickle'
tagline_pickle_file = 'tagline_data.pickle'
summary_pickle_file = 'summary_data.pickle'
string_pickle_file = 'string_data.pickle'
data_pickle_file = 'data.pickle'

# Export each variable to its respective pickle file
with open(story_pickle_file, 'wb') as pickle_file:
    pickle.dump(story, pickle_file)

with open(genre_pickle_file, 'wb') as pickle_file:
    pickle.dump(genre, pickle_file)

with open(tagline_pickle_file, 'wb') as pickle_file:
    pickle.dump(tagline, pickle_file)

with open(summary_pickle_file, 'wb') as pickle_file:
    pickle.dump(summary, pickle_file)

with open(string_pickle_file, 'wb') as pickle_file:
    pickle.dump(string, pickle_file)

with open(data_pickle_file, 'wb') as pickle_file:
    pickle.dump(data, pickle_file)